# *0. Contexte*

BGE est un modèle performant pour la recherche documentaire et dans le classement des bases de données massives. Il est principalement utilisé dans la recherche scientifique, la finance, et la pharmacie pour organiser et classer de grands volumes d’information.

LegalKit contient des paires query ↔ document prêtes pour l’entraînement d’embeddings en droit français (≈53k lignes, licence CC-BY-4.0).
Colonnes typiques : query (question), input (passage légal), + métadonnées.

# 1. *Importation de librairies*

*Pour pouvoir enregistrer une copie sur Github, par la suite*

**A LANCER SEULEMENT UNE FOIS !**

In [ ]:
%pip install -q nbconvert
!jupyter nbconvert --ClearWidgetsPreprocessor.enabled=True \
  --ClearOutputPreprocessor.enabled=True \
  --to notebook --inplace "/content/PFE_BGE_fine_tunning.ipynb"

[NbConvertApp] WARNING | pattern '/content/PFE_BGE_fine_tunning.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=T

In [ ]:
pip install -U FlagEmbedding

In [ ]:
pip install -U datasets sentence-transformers faiss-cpu accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0

# 2. *Dataset*


In [ ]:
from datasets import load_dataset
ds = load_dataset("louisbrulenaudet/legalkit")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/25.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/53000 [00:00<?, ? examples/s]

# 3. *Nettoyage & mapping colonnes → format QA (query, positive)*

In [ ]:
from datasets import DatasetDict
import pandas as pd

# 1. On garde les colonnes dont on a besoin
ds_legalkit = ds["train"].select_columns(["query", "input", "output"])
ds_legalkit

# 2) Concaténer input + output -> "positive" (avec nettoyage + limite longueur)
ds_legalkit = ds_legalkit.map(
    lambda ex: {"query": ex["query"],
                "positive": f"{ex['input'].strip()}, {ex['output'].strip()}"},
    remove_columns=["input", "output"]
)
#2.1 Afficher un aperçu (5 lignes)
pd.set_option("display.max_colwidth", 200)
display(ds_legalkit.select(range(5)).to_pandas())

# 3. enlever lignes vides
nb_vides = sum(1 for ex in ds_legalkit if not ex["query"] or not ex["positive"])
print("\nNombre de lignes vides : ", nb_vides)

# 4. Splitter en 70/15/15
ds_legalkit_dev  = ds_legalkit.train_test_split(test_size=0.30, seed=42, shuffle=True)  # 70% / 30%
ds_legalkit_test = ds_legalkit_dev["test"].train_test_split(test_size=0.5, seed=42, shuffle=True)  # 15% / 15%

splits = DatasetDict({
    "train": ds_legalkit_dev["train"],
    "dev":   ds_legalkit_test["train"],   # (= validation)
    "test":  ds_legalkit_test["test"],
})

# 5. Vérifier les tailles
print("\n", splits)
for name in ["train","dev","test"]:
    print("\n", name + " :", len(splits[name]))

Map:   0%|          | 0/53000 [00:00<?, ? examples/s]

,query,positive
0,"Quelles sont les formalités légales à respecter pour la liquidation et le partage du régime matrimonial lors d'une instance en divorce, notamment lorsque des biens soumis à la publicité foncière s...","Code civil, art. 265-2, Les époux peuvent, pendant l'instance en divorce, passer toutes conventions pour la liquidation et le partage de leur régime matrimonial. Lorsque la liquidation porte sur d..."
1,"Quelles sont les conditions nécessaires pour que les délibérations d'un conseil d'établissement deviennent exécutoires, en ce qui concerne l'affichage, la notification et la transmission des délib...","Code de l'action sociale et des familles, art. R315-23-5, Les délibérations du conseil des établissements mentionnés au présent chapitre ne deviennent exécutoires, dans les conditions fixées soit ..."
2,"Quel est le montant du cautionnement que doit fournir l'exploitant d'un magasin général en fonction de la surface affectée au magasinage, et quels sont les établissements soumis à cette même oblig...","Code de commerce, art. L522-12, L'arrêté préfectoral autorisant l'ouverture du magasin général soumet son exploitant à l'obligation d'un cautionnement. Sont soumis à la même obligation les établis..."
3,"Quels sont les critères pris en compte pour attribuer des aides financières à la distribution à l'étranger d'œuvres représentatives des cinématographies du monde, notamment en ce qui concerne l'ex...","Code du cinéma et de l'image animée, art. 723-6, I. - Les aides financières à la distribution à l'étranger des œuvres représentatives des cinématographies du monde sont attribuées en considération..."
4,"Quels sont les éléments que doivent contenir les statuts et règlements des caisses de congés payés, ainsi que les pièces justificatives et garanties à fournir pour obtenir l'agrément ou fonctionne...","Code du travail, art. D3141-17, Un arrêté du ministre chargé du travail fixe les pièces justificatives, les garanties à fournir par les caisses de congés payés soit en vue de leur agrément, soit a..."



Nombre de lignes vides :  0

 DatasetDict({
    train: Dataset({
        features: ['query', 'positive'],
        num_rows: 37100
    })
    dev: Dataset({
        features: ['query', 'positive'],
        num_rows: 7950
    })
    test: Dataset({
        features: ['query', 'positive'],
        num_rows: 7950
    })
})

 train : 37100

 dev : 7950

 test : 7950


# 4. *Zero-shot avec BGE-M3 (+ Weights & Biases)*

In [ ]:
pip install wandb

In [ ]:
pip install -U tqdm

4.1 Vérifier que Colab utilise bien le GPU (et pas le CPU)

In [ ]:
import torch, transformers, sentence_transformers
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    raise SystemExit("⚠️ Pas de GPU actif. Va dans Runtime > Change runtime type > GPU puis redémarre.")

torch: 2.8.0+cu126
cuda available: True
GPU: Tesla T4


4.2 Petites optimisations globales (GPU + tokenizers)

In [ ]:
import os, torch
os.environ["TOKENIZERS_PARALLELISM"] = "true"
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

4.3 Charger le modèle sur GPU et réduire un peu la longueur max

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "BAAI/bge-m3"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(MODEL_NAME, device=device)

# raccourcir un peu pour accélérer l'encodage (optionnel, bon pour un baseline)
model.max_seq_length = 384  # 512 par défaut. 384 = +rapide, souvent même score

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

4.4 Fonction d’encodage

In [ ]:
import time, numpy as np, os

CACHE_DIR = "./cache_zero_shot_test"
os.makedirs(CACHE_DIR, exist_ok=True)

def encode_texts(texts, tag, use_cache=True):
    cache_file = os.path.join(CACHE_DIR, f"{tag}_{len(texts)}.npy")  # inclut la taille -> pas de mismatch
    if use_cache and os.path.exists(cache_file):
        emb = np.load(cache_file)
        if emb.shape[0] == len(texts):
            print(f"[cache] loaded: {cache_file}")
            return emb

    bs = 256 if device=="cuda" else 32   # T4 tient souvent 256 sur BGE-M3; ajuste si OOM
    t0 = time.time()
    emb = model.encode(
        texts,
        batch_size=bs,
        normalize_embeddings=True,
        show_progress_bar=True,
        convert_to_numpy=True,   # évite copies
    )
    if use_cache:
        np.save(cache_file, emb)
        print(f"[cache] saved: {cache_file} ({emb.shape}) in {time.time()-t0:.1f}s")
    return emb

# *5. Ré-exécuter le zéro-shot avec GPU*

*5.1 Setup commun (modèle + données)*

In [ ]:
import math, os, numpy as np, torch, faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# ----- Modèle -----
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
model = SentenceTransformer("BAAI/bge-m3", device=device)
model.max_seq_length = 384  # un peu plus rapide que 512 pour un baseline
bs = 256 if device=="cuda" else 32

# ----- Données (TEST only) -----
test_q = list(splits["test"]["query"])
test_p = list(splits["test"]["positive"])

print("#queries:", len(test_q))

device: cuda
#queries: 7950


*5.2 Pairwise — “query vs son positive”*

In [ ]:
# 1) Embeddings alignés (même ordre)
q_emb = model.encode(test_q, batch_size=bs, normalize_embeddings=True, show_progress_bar=True)
p_emb = model.encode(test_p, batch_size=bs, normalize_embeddings=True, show_progress_bar=True)

# 2) Cosine pairwise (produit scalaire car normalisé)
cos_pos = (q_emb * p_emb).sum(axis=1)

print("Pairwise cosine:")
print("  mean:", float(np.mean(cos_pos)))
print("  median:", float(np.median(cos_pos)))
print("  min/max:", float(np.min(cos_pos)), float(np.max(cos_pos)))

# 3) Petit sanity-check avec un négatif aléatoire (1 par query)
rng = np.random.default_rng(123)
neg_idx = rng.integers(0, len(test_p), size=len(test_p))
# évite de choisir le gold lui-même
neg_idx = np.where(neg_idx == np.arange(len(test_p)), (neg_idx+1) % len(test_p), neg_idx)
neg_emb = p_emb[neg_idx]

cos_neg = (q_emb * neg_emb).sum(axis=1)

pairwise_acc = float(np.mean(cos_pos > cos_neg))  # % de queries où le positif > négatif
print("Pairwise accuracy (pos > 1 random neg):", round(pairwise_acc, 3))

# 4) (Option) mini-retrieval local pour chaque query avec 1 positif + N négatifs aléatoires
N_NEG = 19  # 1 positif + 19 négatifs => top-20
hits = 0
for i in range(len(test_q)):
    cand_idx = set([i])
    while len(cand_idx) < N_NEG+1:
        j = int(rng.integers(0, len(test_p)))
        if j != i:
            cand_idx.add(j)
    cand_idx = list(cand_idx)
    cand_emb = p_emb[cand_idx]                          # [N, d]
    sims = cand_emb @ q_emb[i]                          # [N]
    # rang du vrai positif (son index local dans cand_idx)
    true_local = cand_idx.index(i)
    rank = (np.argsort(-sims).tolist()).index(true_local) + 1
    hits += (rank == 1)

mini_retrieval_R1 = hits/len(test_q)
print(f"Mini-retrieval@1 (1 pos + {N_NEG} neg aléatoires):", round(mini_retrieval_R1, 3))

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Pairwise cosine:
  mean: 0.7652522921562195
  median: 0.769821286201477
  min/max: 0.29724279046058655 0.9328734278678894
Pairwise accuracy (pos > 1 random neg): 0.999
Mini-retrieval@1 (1 pos + 19 neg aléatoires): 0.997


*5.3 Retrieval réaliste — FAISS sur un corpus (métriques IR)*

In [ ]:
# 1) Corpus = toutes les positives du test, dédupliquées
corpus = list(set(test_p))

# 2) Encodage corpus + queries (réutilise q_emb si identique)
corpus_emb = model.encode(corpus, batch_size=bs, normalize_embeddings=True, show_progress_bar=True)
query_emb  = q_emb  # déjà calculé au-dessus; sinon: model.encode(test_q, ...)

# 3) FAISS index (cosine via inner product sur vecteurs normalisés)
index = faiss.IndexFlatIP(corpus_emb.shape[1])
index.add(corpus_emb)
D, I = index.search(query_emb, 10)  # indices top-10 dans 'corpus' pour chaque query

# 4) Associer chaque gold (test_p[i]) à son index dans 'corpus' (via normalisation de texte)
def norm(s: str) -> str:
    return " ".join(s.split()).strip()

corpus_norm = [norm(x) for x in corpus]
gold_map = {corpus_norm[i]: i for i in range(len(corpus))}
gold_indices = [gold_map.get(norm(g), -1) for g in test_p]  # -1 si absent (ça ne devrait pas arriver)

# 5) Métriques IR
def recall_at_k_idx(I, k, golds):
    hits, total = 0, 0
    for row_idx, gi in enumerate(golds):
        if gi == -1:
            continue
        total += 1
        if gi in I[row_idx, :k]:
            hits += 1
    return hits / max(1,total)

def mrr_at_k_idx(I, k, golds):
    tot, n = 0.0, 0
    for row_idx, gi in enumerate(golds):
        if gi == -1:
            continue
        n += 1
        row = list(I[row_idx, :k])
        if gi in row:
            tot += 1.0 / (row.index(gi) + 1)
    return tot / max(1,n)

def ndcg_at_k_idx(I, k, golds):
    tot, n = 0.0, 0
    for row_idx, gi in enumerate(golds):
        if gi == -1:
            continue
        n += 1
        gains = [1.0 if j == gi else 0.0 for j in I[row_idx, :k]]
        dcg = sum(g / math.log2(i + 2) for i, g in enumerate(gains))
        tot += dcg  # IDCG = 1 (un seul pertinent)
    return tot / max(1,n)

R1   = recall_at_k_idx(I, 1,  gold_indices)
R5   = recall_at_k_idx(I, 5,  gold_indices)
R10  = recall_at_k_idx(I, 10, gold_indices)
MRR10  = mrr_at_k_idx(I, 10, gold_indices)
NDCG10 = ndcg_at_k_idx(I, 10, gold_indices)

print({
    "R@1": round(R1,3), "R@5": round(R5,3), "R@10": round(R10,3),
    "MRR@10": round(MRR10,3), "nDCG@10": round(NDCG10,3),
    "n_queries": len(test_q), "n_docs": len(corpus)
})

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

{'R@1': 0.956, 'R@5': 0.989, 'R@10': 0.993, 'MRR@10': 0.971, 'nDCG@10': 0.976, 'n_queries': 7950, 'n_docs': 7947}


Concrètement, pour bien comprendre :

Pairwise : “le modèle met-il bien la query proche de son positive ?” (cosines, acc vs neg aléatoire).

Retrieval : “parmi des centaines/milliers de passages, la bonne réponse ressort-elle dans le top-k ?”.

*5.4 Weights & Biases (R@1/5/10, MRR, nDCG@10)*

In [ ]:
import wandb
wandb.login()
run = wandb.init(project="legal-embeddings", name="bge-m3_zero-shot_test_pairwise+retrieval")

wandb.log({
  "pairwise/mean_cos": float(np.mean(cos_pos)),
  "pairwise/acc_pos>neg": pairwise_acc,
  "mini_retrieval/R@1_(1pos+19neg)": mini_retrieval_R1,
  "retrieval/R@1": R1, "retrieval/R@5": R5, "retrieval/R@10": R10,
  "retrieval/MRR@10": MRR10, "retrieval/nDCG@10": NDCG10,
  "n_queries": len(test_q), "n_docs": len(corpus)
})
run.finish()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dlaouedj (dlaouedj-ece) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


mini_retrieval/R@1_(1pos+19neg),▁
n_docs,▁
n_queries,▁
pairwise/acc_pos>neg,▁
pairwise/mean_cos,▁
retrieval/MRR@10,▁
retrieval/R@1,▁
retrieval/R@10,▁
retrieval/R@5,▁
retrieval/nDCG@10,▁
mini_retrieval/R@1_(1pos+19neg),0.99723


# 6. Fine-tuning avec LoRA

6.1 Prépare les données + évaluateur (dev)

In [ ]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses, evaluation
import numpy as np, math, faiss, torch

# 1) Créer les exemples d'entraînement (in-batch negatives)
train_examples = [InputExample(texts=[q, p])
                  for q, p in zip(splits["train"]["query"], splits["train"]["positive"])]

# 2) DataLoader
BATCH_SIZE = 64  # baisse à 32 si OOM
train_loader = DataLoader(train_examples, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# 3) Evaluator "IR" sur DEV (retrieval réaliste)
#    On crée un mini-corpus dev et on mappe queries -> passage id
def build_ir_evaluator_from_dev(splits_dev, name="dev-ir"):
    # Corpus = uniques des positives
    corpus = list(set(splits_dev["positive"]))
    # Dictionnaires au format attendu par InformationRetrievalEvaluator
    corpus_dict = {str(i): c for i, c in enumerate(corpus)}
    queries_dict = {str(i): q for i, q in enumerate(splits_dev["query"])}

    # Relevant docs: pour chaque query i, doc id correspondant dans le corpus
    def norm(s): return " ".join(s.split()).strip()
    inv = {norm(c): str(i) for i, c in enumerate(corpus)}
    relevant_docs = {}
    miss = 0
    for i, gold in enumerate(splits_dev["positive"]):
        gi = inv.get(norm(gold))
        if gi is None:
            miss += 1
        else:
            relevant_docs[str(i)] = {gi: 1}
    if miss:
        print(f"[dev evaluator] {miss} gold non retrouvés dans le corpus (après normalisation).")

    # Evaluator avec métriques Recall@k, MAP, MRR, NDCG etc.
    return evaluation.InformationRetrievalEvaluator(
        queries=queries_dict,
        corpus=corpus_dict,
        relevant_docs=relevant_docs,
        show_progress_bar=True,
        mrr_at_k=[10],
        ndcg_at_k=[10],
        accuracy_at_k=[1,5,10],
        recall_at_k=[1,5,10],
        map_at_k=[10],
        name=name
    )

dev_evaluator = build_ir_evaluator_from_dev(splits["dev"], name="dev-ir")

6.2 Charger BGE-M3 + activer LoRA

In [ ]:
import os
from sentence_transformers import SentenceTransformer

MODEL_NAME = "BAAI/bge-m3"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(MODEL_NAME, device=device)
model.max_seq_length = 384   # plus rapide; remets 512 pour le run “officiel”

# --- Activer LoRA via PEFT (option recommandé) ---
USE_LORA = True  # mets False pour fine-tuning "classique" si tu veux comparer

if USE_LORA:
    try:
        from peft import LoraConfig, get_peft_model, TaskType
        # Règle LoRA: r/alpha/dropout — tu peux ajuster
        lora_cfg = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,  # embeddings
            r=16, lora_alpha=32, lora_dropout=0.1,
            # Cible des modules linéaires des blocs d'attention/FFN.
            # "all-linear" marche bien quand on ne connait pas les noms exacts.
            target_modules="all-linear"
        )
        # Récupérer le backbone HF et l'envelopper avec PEFT
        hf_backbone = model._first_module().auto_model
        peft_backbone = get_peft_model(hf_backbone, lora_cfg)
        peft_backbone.print_trainable_parameters()
        # Remettre le backbone LoRA dans SentenceTransformer
        model._first_module().auto_model = peft_backbone
        print("✅ LoRA activé via PEFT.")
    except Exception as e:
        print("⚠️ Impossible d'activer LoRA, on passe en FT classique. Raison:", repr(e))
        USE_LORA = False

# Perfs GPU
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

6.3 Définir la loss et les hyperparamètres, lancer l’entraînement

In [ ]:
from sentence_transformers import losses

# Loss in-batch negatives: pour chaque pair (q, p), les autres p du batch jouent le rôle de négatifs
train_loss = losses.MultipleNegativesRankingLoss(model)

EPOCHS = 1              # commence à 1; passe à 2–3 si tu as le temps
LR = 2e-5               # 2e-5/3e-5 sont de bons points de départ
WARMUP_RATIO = 0.1      # 10% des steps
EVAL_STEPS = 500        # éval dev régulière

# Calcul des steps
num_train_steps = (len(train_loader) * EPOCHS)
warmup_steps = int(num_train_steps * WARMUP_RATIO)

OUTPUT_DIR = "./bge-m3-legalkit-lora" if USE_LORA else "./bge-m3-legalkit-ft"

# (Optionnel) W&B pour tracer la courbe de loss + scores dev
import wandb
USE_WANDB = True
if USE_WANDB:
    wandb.login()
    run = wandb.init(
        project="legal-embeddings",
        name=("bge-m3_lora_ft_v1" if USE_LORA else "bge-m3_ft_v1"),
        group="bge-m3_legalkit",
        config={
            "model": MODEL_NAME,
            "use_lora": USE_LORA,
            "epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
            "lr": LR,
            "warmup_ratio": WARMUP_RATIO,
            "max_seq_len": model.max_seq_length
        }
    )

# Callback simple pour logger la loss step sur W&B
def wandb_callback(score, epoch, steps):
    if USE_WANDB:
        if isinstance(score, dict):
            wandb.log({f"dev/{k}": v for k, v in score.items()}, step=steps)
        else:
            wandb.log({"dev/score": score}, step=steps)

# Entraînement
model.fit(
    train_objectives=[(train_loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    scheduler="cosine",
    optimizer_params={"lr": LR},
    show_progress_bar=True,
    use_amp=True,                 # mixed-precision sur GPU
    evaluator=dev_evaluator,      # évalue sur dev pendant l'entraînement
    evaluation_steps=EVAL_STEPS,
    output_path=OUTPUT_DIR,
    save_best_model=True,
    callback=wandb_callback if USE_WANDB else None
)

if USE_WANDB:
    run.finish()

6.4 Ré-évaluer après entraînement sur le TEST (mêmes métriques que zéro-shot)

In [ ]:
# 1) Charger le modèle finetuné
ft_model_path = OUTPUT_DIR  # celui qu'on vient d'entraîner (best model)
ft_model = SentenceTransformer(ft_model_path, device=device)
ft_model.max_seq_length = 384

# 2) Construire corpus et queries du TEST
test_q = list(splits["test"]["query"])
test_p = list(splits["test"]["positive"])
corpus  = list(set(test_p))

# 3) Encodage
bs = 256 if device=="cuda" else 32
corpus_emb = ft_model.encode(corpus, batch_size=bs, normalize_embeddings=True, show_progress_bar=True)
query_emb  = ft_model.encode(test_q, batch_size=bs, normalize_embeddings=True, show_progress_bar=True)

# 4) FAISS search
index = faiss.IndexFlatIP(corpus_emb.shape[1]); index.add(corpus_emb)
_, I = index.search(query_emb, 10)

# 5) Gold indices + métriques (comme avant)
def norm(s): return " ".join(s.split()).strip()
corpus_norm = [norm(x) for x in corpus]
gold_map = {corpus_norm[i]: i for i in range(len(corpus))}
golds = [gold_map.get(norm(g), -1) for g in test_p]

def recall_at_k(I, k):
    hits=0; total=0
    for r, gi in enumerate(golds):
        if gi==-1: continue
        total+=1
        if gi in I[r,:k]: hits+=1
    return hits/max(1,total)

def mrr_at_k(I, k=10):
    tot=0.0; n=0
    for r,gi in enumerate(golds):
        if gi==-1: continue
        n+=1; row=list(I[r,:k])
        if gi in row: tot += 1.0/(row.index(gi)+1)
    return tot/max(1,n)

def ndcg_at_k(I, k=10):
    tot=0.0; n=0
    for r,gi in enumerate(golds):
        if gi==-1: continue
        n+=1
        gains=[1.0 if j==gi else 0.0 for j in I[r,:k]]
        import math
        dcg=sum(g/math.log2(i+2) for i,g in enumerate(gains))
        tot+=dcg
    return tot/max(1,n)

R1, R5, R10 = recall_at_k(I,1), recall_at_k(I,5), recall_at_k(I,10)
MRR10, NDCG10 = mrr_at_k(I,10), ndcg_at_k(I,10)

print({"FT/R@1":round(R1,3),"FT/R@5":round(R5,3),"FT/R@10":round(R10,3),
       "FT/MRR@10":round(MRR10,3),"FT/nDCG@10":round(NDCG10,3)})